# Introduction to `GeoGraph`

---

## 0. Setup and Loading package

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
import ipyleaflet

from src import geograph
from src.visualisation import geoviewer
from src.constants import UTM35N

---

## 1. Loading Data

In [ ]:
import pathlib
import geopandas as gpd

# Loading Polesia data set from shared Jasmin folder
GWS_DATA_DIR = pathlib.Path(
    "/gws/nopw/j04/ai4er/guided-team-challenge/2021/biodiversity"
)
data_path = GWS_DATA_DIR / "chernobyl_habitat_data" / "Biotope_EUNIS_ver1.shp"
gdf = gpd.read_file(data_path)

# Looking at south-west corner of data
# Choosen because are of wilderness as discussed
minx, miny, maxx, maxy = gdf.total_bounds
square_len = 25000

gdf = gdf.cx[ minx:minx+square_len  ,  miny:miny+square_len]

print("Number of patches in region of interest:", len(gdf))
gdf.plot()
gdf.head(5)

---

## 2. Creating `GeoGraph`

In [ ]:
# Building the main graph structure
graph = geograph.GeoGraph(gdf, 
                          crs=UTM35N, 
                          columns_to_rename={"Eunis_name":"class_label","AREA":"area"})

---

## 3. Creating Habitats

In [ ]:
# First selecting the classes that make up our habitat
# We chose all classes with 'pine' in the name.
pine_classes = [label for label in graph.df.class_label.unique() if 'pine' in label]
pine_classes

In [ ]:
# Proposed distance from email: mobile (<100m), semi mobile (<25m) and sessile (<5m)
graph.add_habitat('Sessile', 
                  max_travel_distance=5, 
                  valid_classes=pine_classes)

graph.add_habitat('Semi mobile', 
                  max_travel_distance=25, 
                  valid_classes=pine_classes)

graph.add_habitat('Mobile',
                  max_travel_distance=500, 
                  valid_classes=pine_classes)

---

## 4. Interactive Graph

In [ ]:
viewer = geoviewer.GeoGraphViewer(small_screen=True)
viewer.add_layer(ipyleaflet.basemaps.Esri.WorldImagery)
viewer.add_graph(graph, name='Polesia data')
viewer.enable_graph_controls()
viewer

### Things to consider in GeoGraphViewer
- How `max_travel_distance` affects the number of components in habitat

## 5. Next Steps

See questions doc...